IMMPORTING THE DATA

In [1]:
import pandas as pd
import numpy as np


In [2]:
file_path = ("/Users/lalitaneeharikavajjhala/Desktop/KAARA/sales_data_sample 2.csv")
df = pd.read_csv(file_path, encoding='latin1')

PRE PROCESSING

In [3]:
df['ORDERDATE'] = pd.to_datetime(df['ORDERDATE'])
# monday is 0 and sunday is 6 

In [4]:
df['day_of_week'] = df['ORDERDATE'].dt.dayofweek 
df['week_of_year'] = df['ORDERDATE'].dt.isocalendar().week
df.head()

,ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,QTR_ID,MONTH_ID,YEAR_ID,...,CITY,STATE,POSTALCODE,COUNTRY,TERRITORY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE,day_of_week,week_of_year
0,10107,30,95.70,2,2871.00,2003-02-24,Shipped,1,2,2003,...,NYC,NY,10022,USA,NaN,Yu,Kwai,Small,0,9
1,10121,34,81.35,5,2765.90,2003-05-07,Shipped,2,5,2003,...,Reims,NaN,51100,France,EMEA,Henriot,Paul,Small,2,19
2,10134,41,94.74,2,3884.34,2003-07-01,Shipped,3,7,2003,...,Paris,NaN,75508,France,EMEA,Da Cunha,Daniel,Medium,1,27
3,10145,45,83.26,6,3746.70,2003-08-25,Shipped,3,8,2003,...,Pasadena,CA,90003,USA,NaN,Young,Julie,Medium,0,35
4,10159,49,100.00,14,5205.27,2003-10-10,Shipped,4,10,2003,...,San Francisco,CA,NaN,USA,NaN,Brown,Julie,Medium,4,41


In [5]:
# cyclical encoding to preserve the cyclic and periodic nature of the elements
df['month_sin'] = np.sin(2 * np.pi * df['MONTH_ID'] / 12)
df['month_cos'] = np.cos(2 * np.pi * df['MONTH_ID'] / 12)
df['quarter_sin'] = np.sin(2 * np.pi * df['QTR_ID'] / 4)
df['quarter_cos'] = np.cos(2 * np.pi * df['QTR_ID'] / 4)

In [6]:
drop_columns = ['ORDERNUMBER', 'ORDERDATE', 'CUSTOMERNAME', 'PHONE', 'ADDRESSLINE1', 'ADDRESSLINE2', 'CONTACTLASTNAME', 'CONTACTFIRSTNAME', 'POSTALCODE']
df = df.drop(columns=drop_columns)

In [7]:
# Handle missing values
df['STATE'].fillna(df['STATE'].mode()[0], inplace=True)  # Impute missing values with mode
# df['POSTALCODE'].fillna(df['POSTALCODE'].median(), inplace=True)  # Impute missing values with median
df['TERRITORY'].fillna('Unknown', inplace=True)  # Impute missing values with a placeholder (assuming 'Unknown')

/var/folders/lm/8dmqs1pd6qg4vn2y7m90m3pw0000gn/T/ipykernel_2277/1219511153.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['STATE'].fillna(df['STATE'].mode()[0], inplace=True)  # Impute missing values with mode
/var/folders/lm/8dmqs1pd6qg4vn2y7m90m3pw0000gn/T/ipykernel_2277/1219511153.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate ob

In [8]:
# performing one hot encoding for the categorical columns
df = pd.get_dummies(df, columns=['STATUS', 'PRODUCTLINE', 'PRODUCTCODE', 'CITY', 'STATE', 'COUNTRY', 'TERRITORY', 'DEALSIZE'])

In [9]:
# defining the target variable
y = df['SALES']
X = df.drop(columns=['SALES'])

In [10]:
X.head()

,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,QTR_ID,MONTH_ID,YEAR_ID,MSRP,day_of_week,week_of_year,month_sin,...,COUNTRY_Switzerland,COUNTRY_UK,COUNTRY_USA,TERRITORY_APAC,TERRITORY_EMEA,TERRITORY_Japan,TERRITORY_Unknown,DEALSIZE_Large,DEALSIZE_Medium,DEALSIZE_Small
0,30,95.70,2,1,2,2003,95,0,9,0.866025,...,False,False,True,False,False,False,True,False,False,True
1,34,81.35,5,2,5,2003,95,2,19,0.500000,...,False,False,False,False,True,False,False,False,False,True
2,41,94.74,2,3,7,2003,95,1,27,-0.500000,...,False,False,False,False,True,False,False,False,True,False
3,45,83.26,6,3,8,2003,95,0,35,-0.866025,...,False,False,True,False,False,False,True,False,True,False
4,49,100.00,14,4,10,2003,95,4,41,-0.866025,...,False,False,True,False,False,False,True,False,True,False


In [11]:
X.dtypes

QUANTITYORDERED        int64
PRICEEACH            float64
ORDERLINENUMBER        int64
QTR_ID                 int64
MONTH_ID               int64
                      ...   
TERRITORY_Japan         bool
TERRITORY_Unknown       bool
DEALSIZE_Large          bool
DEALSIZE_Medium         bool
DEALSIZE_Small          bool
Length: 250, dtype: object

In [12]:
non_numeric_columns = df.select_dtypes(exclude=['number']).columns
print(f"Non-numeric columns: {non_numeric_columns}")

Non-numeric columns: Index(['STATUS_Cancelled', 'STATUS_Disputed', 'STATUS_In Process',
       'STATUS_On Hold', 'STATUS_Resolved', 'STATUS_Shipped',
       'PRODUCTLINE_Classic Cars', 'PRODUCTLINE_Motorcycles',
       'PRODUCTLINE_Planes', 'PRODUCTLINE_Ships',
       ...
       'COUNTRY_Switzerland', 'COUNTRY_UK', 'COUNTRY_USA', 'TERRITORY_APAC',
       'TERRITORY_EMEA', 'TERRITORY_Japan', 'TERRITORY_Unknown',
       'DEALSIZE_Large', 'DEALSIZE_Medium', 'DEALSIZE_Small'],
      dtype='object', length=237)


In [13]:
missing_values = df.isnull().sum()
print(f"Missing values:\n{missing_values}")

Missing values:
QUANTITYORDERED      0
PRICEEACH            0
ORDERLINENUMBER      0
SALES                0
QTR_ID               0
                    ..
TERRITORY_Japan      0
TERRITORY_Unknown    0
DEALSIZE_Large       0
DEALSIZE_Medium      0
DEALSIZE_Small       0
Length: 251, dtype: int64


BUILDING THE ML MODEL

In [14]:
# Splitting the data into training and testing sets
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [15]:
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

In [16]:
# Predicting the output of the model
y_pred = rf_model.predict(X_test)
print(y_pred)

[1513.5717 2161.0872 4548.2991 4022.9767 3160.412  8725.6807 8576.1625
 1662.7667 2332.5356 8932.0443 2648.1267 2146.5348  741.5587 4677.7741
 5583.9218 5622.3428 1583.9796 5547.3546 2287.0856 1011.9393 5020.4367
 3176.1856 8071.2378 1542.6729 1471.3246 8004.9851  898.2533 5391.4019
 3686.5989 2518.25   3359.9382 4438.0298 2846.7253 2576.3557 3960.1327
  936.5081 1575.1405 2304.3249 3173.1343 3277.043  5340.281  3181.0604
 3662.1127 1270.6513 3598.5416 3654.3656 4198.4941 3619.2678 4137.5887
 1892.9901  828.4196 4563.85   2398.9411 1902.5936 2717.0756  817.3836
 1097.3511  962.8457 5046.2247 2209.4461 1475.5468 4086.3889 1390.5905
 5674.1227 3840.7584 5332.2528 2251.0971 2537.8136 3263.8461 2379.096
 3486.7108 8167.697  4980.2664 5748.0506 1415.9654 1477.545  2847.5319
 4973.2563 4542.6306 4641.1262 3182.5285 2866.4564 3493.8603 7866.5428
 3786.632  5056.8246 5711.1204 3234.5362 3244.4687 5241.6663 2799.7155
 3663.3073 7647.6368 4399.3232 2406.0196 1512.8868 4750.1766 5568.6087
 3888.5

In [20]:
average_sales = np.mean(y_pred)
print(f"Average predicted sales: {average_sales}")

total_sales_2006 = np.sum(y_pred)
print(f"Total predicted sales for 2006: {total_sales_2006}")

Average predicted sales: 3583.2462012389383
Total predicted sales for 2006: 2024534.1037


EVALUATION

In [17]:
# Evaluating the model
from sklearn.metrics import mean_absolute_error, mean_squared_error
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

print(f"MAE: {mae}, RMSE: {rmse}")

MAE: 284.8403506194689, RMSE: 611.8868123946831


In [18]:
# Hyperparameter Tuning
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [100, 200, 300, 400],
    'max_depth': [None, 10, 20, 30, 40],
    'min_samples_split': [2, 5, 10, 15],
    'min_samples_leaf': [1, 2, 4, 6]
}

# Grid Search
grid_search = GridSearchCV(estimator=RandomForestRegressor(random_state=42),
                           param_grid=param_grid,
                           scoring='neg_mean_absolute_error',
                           cv=3,
                           verbose=2,
                           n_jobs=-1)
grid_search.fit(X_train, y_train)

# Best parameters and performance
print(f"Best Parameters: {grid_search.best_params_}")
best_rf_model = grid_search.best_estimator_
best_y_pred = best_rf_model.predict(X_test)
best_mae = mean_absolute_error(y_test, best_y_pred)
best_rmse = np.sqrt(mean_squared_error(y_test, best_y_pred))

print(f"Best Model - MAE: {best_mae}, RMSE: {best_rmse}")

Fitting 3 folds for each of 320 candidates, totalling 960 fits


[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   1.5s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   1.5s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   1.5s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   2.7s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   2.9s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   3.0s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   1.1s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   1.2s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=300; total time=   4.4s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estim

EXPORTING AS A NEW CSV

In [21]:
y_pred_series = pd.Series(y_pred, name='PREDICTED_SALES')
df_with_pred = pd.concat([df, y_pred_series], axis=1)

In [22]:
output_file_path = "/Users/lalitaneeharikavajjhala/Desktop/KAARA/sales_data_sample_with_predictions.csv"
df_with_pred.to_csv(output_file_path, index=False)


In [24]:
pred_df = df_with_pred['PREDICTED_SALES']
opfile = "/Users/lalitaneeharikavajjhala/Desktop/KAARA/predictions.csv"
pred_df.to_csv(opfile, index=False)